In [1]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
from scipy import stats 
import matplotlib.pyplot as plt
import os

In [2]:
filepath = '../raw_data/productivity_data/total_factor_productivity.csv'
try:
	disp_df = pd.read_csv(filepath)
except UnicodeDecodeError:
	disp_df = pd.read_csv(filepath, encoding='latin1')

In [5]:
df = disp_df.copy()

In [ ]:
plt.figure(figsize=(12, 6), dpi=120)
ax = plt.subplot(111)
ax.set_facecolor('white')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(which='major', linestyle=':', linewidth=0.7, alpha=0.7)
ax.set_axisbelow(True)

industries = df['naics4'].unique()
for industry in industries:
    subset = df[df['naics4'] == industry]
    plt.plot(subset['year'], subset['d7525*'], 
             alpha=0.3, linewidth=0.8, color='grey')

if 'weight' in df.columns:
    weighted_avg = df.groupby('year').apply(
        lambda x: np.average(x['d7525*'], weights=x['weight'])
    )
    plt.plot(weighted_avg.index, weighted_avg.values, 
             linewidth=3, color='#1f77b4', label='Weighted average dispersion')
else:
    average_by_year = df.groupby('year')['d7525*'].mean()
    plt.plot(average_by_year.index, average_by_year.values, 
             linewidth=3, color='#1f77b4', label='Average dispersion')

plt.xlabel('Year', fontsize=12)
plt.ylabel('Interquartile Range (75-25 percentile difference)', fontsize=12)
plt.title('Industry Dispersion Trends', fontsize=14)
plt.legend()
plt.tight_layout()


plt.savefig('../figures/dispersion_trends.png', 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for col in ['d7525*', 'd9010*', 'd9990*', 'd1001*']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

avg_dispersion = df.groupby('year')[['d7525*', 'd9010*', 'd9990*', 'd1001*']].mean()

normalized = avg_dispersion.div(avg_dispersion.iloc[0])  
plt.figure(figsize=(12, 6), dpi=120)
ax = plt.subplot(111)
ax.set_facecolor('white')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(which='major', linestyle=':', linewidth=0.7, alpha=0.7)
ax.set_axisbelow(True)

plt.plot(normalized.index, normalized['d7525*'], 
         linewidth=2, marker='o', markersize=4, label='75-25 percentile')
plt.plot(normalized.index, normalized['d9010*'], 
         linewidth=2, marker='s', markersize=4, label='90-10 percentile')
plt.plot(normalized.index, normalized['d9990*'], 
         linewidth=2, marker='^', markersize=4, label='99-90 percentile')
plt.plot(normalized.index, normalized['d1001*'], 
         linewidth=2, marker='d', markersize=4, label='10-1 percentile')

plt.xlabel('Year', fontsize=12)
plt.ylabel('Normalized Dispersion (Base Year = 1)', fontsize=12)
plt.title('Comparison of Dispersion Measures Over Time', fontsize=14)
plt.legend()
plt.tight_layout()

plt.savefig('../figures/dispersion_comparison.png', 
            dpi=300, bbox_inches='tight')
plt.show()